<a href="https://colab.research.google.com/github/someshsingh22/Sherlocked/blob/master/Sherlocked.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/someshsingh22/Sherlocked

Cloning into 'Sherlocked'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 54 (delta 14), reused 38 (delta 1), pack-reused 0
Unpacking objects: 100% (54/54), done.


In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from collections import Counter
import os
import re
from argparse import Namespace
import time
from tqdm import tqdm

In [0]:
def predict(device, net, words, n_vocab, vocab_to_int, int_to_vocab, top_k=5):
    net.eval()

    state_h, state_c = net.zero_state(1)
    state_h = state_h.to(device)
    state_c = state_c.to(device)
    for w in words:
        ix = torch.tensor([[vocab_to_int[w]]]).to(device)
        output, (state_h, state_c) = net(ix, (state_h, state_c))
    
    _, top_ix = torch.topk(output[0], k=top_k)
    choices = top_ix.tolist()
    choice = np.random.choice(choices[0])

    words.append(int_to_vocab[choice])
    for _ in range(100):
        ix = torch.tensor([[choice]]).to(device)
        output, (state_h, state_c) = net(ix, (state_h, state_c))

        _, top_ix = torch.topk(output[0], k=top_k)
        choices = top_ix.tolist()
        choice = np.random.choice(choices[0])
        words.append(int_to_vocab[choice])

    return ' '.join(words).encode('utf-8')


In [0]:
 y=predict(device, net, "Sherlock Holmes".split(), n_vocab, vocab_to_int, int_to_vocab, top_k=2)

In [0]:
y

b'Sherlock Holmes had said to my own to do with the first thing that I had seen a more powerfully built young man , and the same necessity was growing in the room , and I was to be able for a few minutes , but he had been a very busy man . He was a very tall , dark man with the broad and ready face . He was a perfect savage , tall , strong , strong object was a familiar man , and he was a very tall and fair haired woman hobbled to the table . He was'

In [0]:
path="./Sherlocked/Dataset/Clean/{}.txt"

Data_flags = Namespace(
    data_dir=path.format("cano"),
    seq_size=64,
    batch_size=256,
)
Brain_flags=Namespace(
    num_layers=1,
    is_bidirectional=False,
    seq_size=Data_flags.seq_size,
    batch_size=Data_flags.batch_size,
    embedding_size=256,
    lstm_size=1024,
    gradients_norm=5,
    dropout=0.1
)
Pen_flags=Namespace(
)

In [0]:
# Data Class
class Data :
  def __init__(self,Data_flags):
    self.flags=Data_flags
    
    # Pre Process Data
    self.pre_process()
    
  # Prepare Data For Training
  def pre_process(self):
    start=time.time()
    # Read datat and split to words from files
    text=open(Data_flags.data_dir).read().split()

    # Create Frequency Dictionary
    word_counts = Counter(text)
    self.sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)

    # Word2Vec Mapping
    self.int_to_vocab = {k: w for k, w in enumerate(self.sorted_vocab)}
    self.vocab_to_int = {w: k for k, w in self.int_to_vocab.items()}
    self.n_vocab = len(self.int_to_vocab)

    # Create Input-Output Data
    self.int_text = [self.vocab_to_int[w] for w in text]
    num_batches = int(len(self.int_text) / (Data_flags.seq_size * Data_flags.batch_size))
    self.in_text = self.int_text[:num_batches * Data_flags.batch_size * Data_flags.seq_size]
    self.out_text = np.zeros_like(self.in_text)
    self.out_text[:-1] = self.in_text[1:]
    self.out_text[-1] = self.in_text[0]
    self.in_text = np.reshape(self.in_text, (Data_flags.batch_size, -1))
    self.out_text = np.reshape(self.out_text, (Data_flags.batch_size, -1))
    print("Data Preprocessing complete with {} words".format(self.n_vocab))

  # Create Input-Output Batch Generator 
  def get_batches(self):
    num_batches = np.prod(self.in_text.shape) // (self.flags.seq_size * self.flags.batch_size)
    for i in range(0, num_batches * self.flags.seq_size, self.flags.seq_size):
      yield self.in_text[:, i:i+self.flags.seq_size], self.out_text[:, i:i+self.flags.seq_size]

In [0]:
# Neural Network
class Network(nn.Module):
  def __init__(self,flags,data):
    super(Network, self).__init__()
    self.flags=flags
    self.data=data

    # Embedding Layer , LSTM Stack and Output Layer
    self.embedding= nn.Embedding(data.n_vocab, self.flags.embedding_size)
    self.lstm=nn.LSTM(self.flags.embedding_size,self.flags.lstm_size,batch_first=True,num_layers=self.flags.num_layers,dropout=self.flags.dropout)
    self.dense = nn.Linear(self.flags.lstm_size, data.n_vocab)

  # Forward Pass
  def forward(self, x, prev_state):
      embed = self.embedding(x)
      output, state = self.lstm(embed, prev_state)
      logits = self.dense(output)
      return logits, state

  # ZeroState Init
  def zero_state(self):
        return (torch.zeros(self.flags.num_layers*(2 if self.flags.is_bidirectional else 1), self.flags.batch_size, self.flags.lstm_size),
                torch.zeros(self.flags.num_layers*(2 if self.flags.is_bidirectional else 1), self.flags.batch_size, self.flags.lstm_size))

# Solver / Optimizer
class Optimizer :
  def __init__(self,Network):
    self.lr=0.02
    self.Network=Network

    # Defining Loss function and Optimizer
    self.criterion = nn.CrossEntropyLoss()
    self.optimizer = torch.optim.Adam(self.Network.parameters(), lr=self.lr)

  def decay(self,loss):

    # Update with Factor or value
    def factor_update(factor):
      for group in self.optimizer.param_groups:
        group['lr']*=value
      self.lr*=value
    def value_update(value):
      for group in self.optimizer.param_groups:
        group['lr']=value
      self.lr=value

    if self.lr > 0.001:
      # Initial Fast Changes
      if loss > 10 :
        self.value_update(0.02)
      elif loss > 7 :
        self.value_update(0.01)
      #slower changes
      elif loss > 4 :
        self.value_update(0.005)
      #exponential drop
      else :
        self.factor_update(0.97)


In [0]:
# Neural Network aka Brain
class Brain :
  def __init__(self,flags,Optimizer):
    self.device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    self.flags=flags
    self.Optimizer=Optimizer
    self.Network=self.Optimizer.Network
    self.data=self.Network.data
    self.Network.to(self.device)
    self.loss_value=0
    
  #Train Function
  def Train(self,epochs):
    self.Network.train()
    for epoch in range(epochs):
      start=time.time()
      batches=self.data.get_batches()
      state_h, state_c = self.Network.zero_state()
      
      
      # Transfer data to GPU
      state_h = state_h.to(self.device)
      state_c = state_c.to(self.device)
      
      for x, y in tqdm(batches):
        
        # Reset all gradients
        self.Optimizer.optimizer.zero_grad()

        # Transfer data to GPU
        x = torch.tensor(x).to(self.device)
        y = torch.tensor(y).to(self.device)

        logits, (state_h, state_c) = self.Network(x, (state_h, state_c))
        self.loss = self.Optimizer.criterion(logits.transpose(1, 2), y)

        state_h = state_h.detach()
        state_c = state_c.detach()

        self.loss_value = self.loss.item()
        
        # Update the network's parameters
        self.Optimizer.optimizer.step()
        self.loss.backward()
        _ = torch.nn.utils.clip_grad_norm_(self.Network.parameters(), self.flags.gradients_norm)
        #self.Optimizer.decay(self.loss)
        
        self.Optimizer.optimizer.step()
      print('Epoch: {}/{}'.format(epoch, epochs),'Loss: {}'.format(self.loss_value),'Time Taken : {}'.format(time.time()-start))

In [0]:
import warnings
warnings.simplefilter("ignore")
d=Data(Data_flags)
n=Network(Brain_flags,d)
o=Optimizer(n)
br=Brain(Brain_flags,o)
br.Train(2)


In [0]:
class Pen :
  def __init__()

11it [00:08,  1.28it/s]


KeyboardInterrupt: ignored

In [231]:
print(data)